In [ ]:
from netdissect import nethook, setting, renormalize, zdataset, paintwidget, labwidget

# Load a generator, and paint random image number 20 in a widget.
G = nethook.InstrumentedModel(setting.load_proggan('church')).cuda()
canvas_z = zdataset.z_sample_for_model(G)[28][None].cuda()
canvas = paintwidget.PaintWidget(image=renormalize.as_url(G(canvas_z)[0]))
LAYERNAME = 'layer4'

# Whenever the widget changes, call "changed" to edit the layer and repaint
TARGET_UNITS = [119, 125, 157, 195, 365, 374, 418, 426, 408, 240]
def canvas_changed(c):
    global TARGET_UNITS
    if not canvas.mask: return
    a = renormalize.from_url(canvas.mask, target='pt', size=(8,8))[0]
    def editrule(x, imodel, **buffers):
        x[:,TARGET_UNITS] += (a * 50.0).to(x.device)
        return x
    G.edit_layer(LAYERNAME, rule=editrule)
    canvas.mask, canvas.image = '', renormalize.as_url(G(canvas_z)[0])
    G.remove_edits()
canvas.on('mask', canvas_changed)

display(canvas)

In [ ]:
probe_z = zdataset.z_sample_for_model(G)[13][None].cuda()
prober = paintwidget.PaintWidget(image=renormalize.as_url(G(probe_z)[0]))
probe_out = labwidget.Div()
display(prober)
display(probe_out)

def probe_changed(c):
    if not prober.mask: return
    a = renormalize.from_url(prober.mask, target='pt', size=(8,8))[0]
    if a.sum() <= 0.0: return
    G.retain_layer(LAYERNAME)
    prober.mask = ''
    activations = G.retained_layer(LAYERNAME)
    mean = (activations.cpu()[0] * a[None]).sum(2).sum(1) / (a.sum() + 1e-8)
    value, order = mean.sort(0, descending=True)
    probe_out.innerHTML = ''
    probe_out.print([o.item() for o in order[:10]])
    probe_out.print([v.item() for v in value[:10]])
    global TARGET_UNITS
    TARGET_UNITS = [o.item() for o in order[:10]]
prober.on('mask', probe_changed)
None